# Yarışma submitlerini 1 ve 0 olarak yapılması yazısına göre yaparak düşük roc-auc score elde ettim. Probablity lerini late submission yaparak denediğimde skorumun yüksek olduğunu gördüm.

In [ ]:
# olasılıkları yüklediğimde aslında yaptıklarımın dogru oldugunu gördüm

<img src="https://i.imgyukle.com/2021/02/21/L2bfKx.png"/>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/turkiye-is-bankas-machine-learning-challenge-3/sample_submission.csv
/kaggle/input/turkiye-is-bankas-machine-learning-challenge-3/monthly_expenditures.csv
/kaggle/input/turkiye-is-bankas-machine-learning-challenge-3/train.csv
/kaggle/input/turkiye-is-bankas-machine-learning-challenge-3/test.csv


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from scipy.stats import norm

from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from collections import Counter
import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')

# Load Data

In [4]:
path = '/kaggle/input/turkiye-is-bankas-machine-learning-challenge-3/'

train_df = pd.read_csv(path+'train.csv', parse_dates=['tarih'])
test_df = pd.read_csv(path+'test.csv', parse_dates=['tarih'])
monthly_exp = pd.read_csv(path+'monthly_expenditures.csv', parse_dates=['tarih'])
sub = pd.read_csv(path+'sample_submission.csv')

for df in [train_df, test_df]:
    df.drop(['tarih','meslek_grubu'], axis=1, inplace=True)
    
y = train_df['target']
train_df.drop('target', axis=1, inplace=True)

# Incorrect values & Some changes

In [5]:
# tarihlerin farkı ay olduğu için ay olarak çeviriyorum
monthly_exp['ay'] = [x.month for x in monthly_exp['tarih']]

monthly_exp.drop('tarih', axis=1, inplace=True)

In [6]:
# tl to dolar
# o aykı dolar kuruyla tl -> dolara çeviriyorum
monthly_exp.loc[monthly_exp['ay'] == 1, 'aylik_toplam_tutar'] = monthly_exp[monthly_exp['ay'] == 1]['aylik_toplam_tutar'] / 5.3759
monthly_exp.loc[monthly_exp['ay'] == 2, 'aylik_toplam_tutar'] = monthly_exp[monthly_exp['ay'] == 2]['aylik_toplam_tutar'] / 5.2769
monthly_exp.loc[monthly_exp['ay'] == 3, 'aylik_toplam_tutar'] = monthly_exp[monthly_exp['ay'] == 3]['aylik_toplam_tutar'] / 5.4666
monthly_exp.loc[monthly_exp['ay'] == 4, 'aylik_toplam_tutar'] = monthly_exp[monthly_exp['ay'] == 4]['aylik_toplam_tutar'] / 5.7617
monthly_exp.loc[monthly_exp['ay'] == 5, 'aylik_toplam_tutar'] = monthly_exp[monthly_exp['ay'] == 5]['aylik_toplam_tutar'] / 6.0560
monthly_exp.loc[monthly_exp['ay'] == 6, 'aylik_toplam_tutar'] = monthly_exp[monthly_exp['ay'] == 6]['aylik_toplam_tutar'] / 5.8127

In [7]:
# islem_adedi: less than 0
abs_islem_adedi = monthly_exp[monthly_exp['islem_adedi'] < 0]['islem_adedi'].apply(lambda x: abs(x))
monthly_exp.loc[monthly_exp['islem_adedi'] < 0, 'islem_adedi'] = abs_islem_adedi

In [8]:
# week to year
train_df['kidem_suresi'] = train_df['kidem_suresi'].apply(lambda x: (x/4.34812141)/12)

In [9]:
# week to year
test_df['kidem_suresi'] = test_df['kidem_suresi'].apply(lambda x: (x/4.34812141)/12)

In [10]:
# fill the kidem_suresi < 0 by mean of the similars
for index in list(train_df[train_df['kidem_suresi'] < 0].index):
    similars_median = np.median(train_df[(train_df['kidem_suresi'] > 0) & (train_df['yas'] == train_df.iloc[index]['yas']) & 
         (train_df['egitim'] == train_df.iloc[index]['egitim']) & (train_df['is_durumu'] == train_df.iloc[index]['is_durumu'])]['kidem_suresi'], axis=0)
    
    train_df.iloc[index, 2] = similars_median

In [11]:
# fill the kidem_suresi < 0 by mean of the similars
for index in list(test_df[test_df['kidem_suresi'] < 0].index):
    similars_median = np.median(train_df[(train_df['kidem_suresi'] > 0) & (train_df['yas'] == train_df.iloc[index]['yas']) & 
         (train_df['egitim'] == train_df.iloc[index]['egitim']) & (train_df['is_durumu'] == train_df.iloc[index]['is_durumu'])]['kidem_suresi'], axis=0)
    
    test_df.iloc[index, 2] = similars_median

In [12]:
# missing values, mode ile dolduruyorum
for df in [train_df, test_df]:  
    df['egitim'].fillna(train_df['egitim'].value_counts().index[0], inplace=True)
    df['is_durumu'].fillna(train_df['is_durumu'].value_counts().index[0], inplace=True)

# Log 

In [13]:
# skewness engellemek için log uyguluyorum
train_log = train_df[['yas']].apply(np.log, axis = 1)
test_log = test_df[['yas']].apply(np.log, axis = 1)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_log)

train_df['yas']= scaler.transform(train_log)
test_df['yas']= scaler.transform(test_log)

In [14]:
# log tanımsız/sonsuz olmaması için kontrol
def neg_to_zero(x):
    if x <= 0:
        return 1
    else:
        return x

In [15]:
train_df['kidem_suresi'] = [neg_to_zero(x) for x in train_df.kidem_suresi]
test_df['kidem_suresi'] = [neg_to_zero(x) for x in test_df.kidem_suresi]

train_log = train_df[['kidem_suresi']].apply(np.log, axis = 1)
test_log = test_df[['kidem_suresi']].apply(np.log, axis = 1)

scaler = StandardScaler()
scaler.fit(train_log)

train_df['kidem_suresi']= scaler.transform(train_log)
test_df['kidem_suresi']= scaler.transform(test_log)

In [16]:
mont_log = monthly_exp[['islem_adedi']].apply(np.log, axis = 1)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(mont_log)

monthly_exp['islem_adedi'] = scaler.transform(mont_log)

In [17]:
monthly_exp['aylik_toplam_tutar'] = [neg_to_zero(x) for x in monthly_exp.aylik_toplam_tutar]

mont_log = monthly_exp[['aylik_toplam_tutar']].apply(np.log, axis = 1)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(mont_log)

monthly_exp['aylik_toplam_tutar'] = scaler.transform(mont_log)

# Join Data

In [18]:
full_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

del train_df
del test_df

In [19]:
deneme_df = pd.DataFrame()

deneme_df['musteri'] = monthly_exp.musteri.unique()

# for i in range(1,7):
#     groupby_ay = monthly_exp[monthly_exp['ay'] == i].groupby('musteri')['islem_adedi','aylik_toplam_tutar'].sum()
#     tutar_islem = groupby_ay['aylik_toplam_tutar'] / groupby_ay['islem_adedi']
#     deneme_df = deneme_df.merge(tutar_islem.rename('ay_'+str(i)), on='musteri', how='left')
    
# deneme_df.fillna(0, inplace=True)
# deneme_df['ortalama_islem_harcama'] = np.mean(deneme_df.loc[:,'ay_1':'ay_6'], axis=1)

# deneme_df.drop(deneme_df.loc[:,'ay_1':'ay_6'].columns, axis=1, inplace=True)

In [20]:
# ay ve sektöre göre harcamaları column'a dönüştürüyorum
for month in range(1,7):
    for feature in list(monthly_exp['sektor'].value_counts().index):
        groupby_ay = monthly_exp[monthly_exp['ay'] == month].groupby(['musteri','sektor'])['islem_adedi','aylik_toplam_tutar'].sum()
        sektor = groupby_ay.xs(feature, level=1, drop_level=False)
        take_sektor_islem_tutar = sektor['aylik_toplam_tutar'] #/ sektor['islem_adedi']
        deneme_df = deneme_df.merge(take_sektor_islem_tutar.rename('ay_'+str(month)+'_'+feature), on='musteri', how='left')
        
deneme_df.fillna(0, inplace=True)

In [21]:
# yaptıgı harcamaya göre bir müşteri type olusturuyorum

# islem_toplam = monthly_exp.groupby(['musteri'])['islem_adedi','aylik_toplam_tutar'].sum()['islem_adedi']
tutar_toplam = monthly_exp.groupby(['musteri'])['islem_adedi','aylik_toplam_tutar'].sum()['aylik_toplam_tutar']
#deneme_df['per_type'] = pd.qcut(tutar_toplam / islem_toplam, 4).values
deneme_df['harcama_type'] = pd.qcut(tutar_toplam, 6).values

In [22]:
# aylik islem adetleri ile yeni columnlar yaratıyorum
for month in range(1,7):
    groupby_ay = monthly_exp[monthly_exp['ay'] == month].groupby(['musteri'])['islem_adedi'].sum()
    deneme_df = deneme_df.merge(groupby_ay.rename('ay_'+str(month)), on='musteri', how='left')
    
deneme_df.loc[:,'ay_1':'ay_6'] = deneme_df.loc[:,'ay_1':'ay_6'].fillna(0)

In [23]:
# yaptıgı isleme göre yeni müsteri type olustuyorum
deneme_df['islem_type'] = deneme_df.loc[:,'ay_1':'ay_6'].sum(axis=1)

deneme_df['islem_type'] = pd.qcut(deneme_df['islem_type'], 6).values

In [24]:
full_df = full_df.merge(deneme_df, on='musteri', suffixes=('_left', '_right'), how='inner')

In [25]:
# son hali
full_df.head()

,musteri,yas,kidem_suresi,egitim,is_durumu,ay_1_RESTORAN_CATER,ay_1_GIYIM_AKSESUAR,ay_1_BENZIN_YAKIT,ay_1_ELKT_ESYA_BILG,ay_1_TASIMACILIK,...,ay_6_KUYUMCU,ay_6_KLP_DERNK_SOSY,harcama_type,ay_1,ay_2,ay_3,ay_4,ay_5,ay_6,islem_type
0,23b2476c8a,2.110481,-0.095449,5f8ca0f83b,915be3a7a4,-0.338302,-0.058719,0.000000,0.000000,0.0,...,0.0,0.0,"(-3.841, -1.946]",-1.593089,0.876684,0.068691,-0.276160,0.508936,0.000000,"(-0.808, 0.0458]"
1,5a7e3a7122,1.607754,1.244046,01a4f2c96c,915be3a7a4,-1.186297,0.000000,0.000000,-0.526230,0.0,...,0.0,0.0,"(-0.84, 0.322]",-1.615986,-0.807993,-0.807993,-1.615986,-1.593089,0.853787,"(-19.563000000000002, -3.371]"
2,2ec95c7499,1.499499,1.184619,7e6640bfe0,d36d84f51a,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,"(-1.946, -0.84]",0.000000,-0.807993,0.000000,0.000000,0.000000,0.000000,"(-1.832, -0.808]"
3,54399dac33,1.035956,0.636536,5f8ca0f83b,991c4998fb,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,"(-1.946, -0.84]",0.000000,0.000000,0.000000,0.000000,-0.807993,0.000000,"(-1.832, -0.808]"
4,97a74b2e58,1.816177,0.834957,7bb291e291,242927d0f5,0.000000,0.000000,0.336609,0.807552,0.0,...,0.0,0.0,"(0.322, 3.248]",-0.762199,0.000000,0.000000,0.000000,0.000000,0.000000,"(-0.808, 0.0458]"


In [26]:
# kidem_suresi içinde -1 değerler var dikkat edilmeli
#full_df[full_df['kidem_suresi'] == -1]['musteri'].isin(idx)

# 4 , 7

In [27]:
# full_df.drop('musteri', axis=1, inplace=True)

# Create model df

In [28]:
model_df = pd.DataFrame()

In [29]:
model_df['yas'] = pd.qcut(full_df['yas'], 6)

model_df['kidem_suresi'] = pd.qcut(full_df['kidem_suresi'], 6)

In [30]:
# sadece bir tane bu eğitimden var onu mode ile değiştir
# fazla column olusturmanın gereksiz olcagını düşündüm
full_df.loc[48736, 'egitim'] = '7e6640bfe0'

In [31]:
# 13 grup var

In [32]:
# grafikler çizerek benzer evlilik oranlarına sahip is durumlarını ortak noktalarda topladım
model_df['is_durumu'] = full_df['is_durumu'].map({'3773727d6e':0, '83a26fc2fd':0, '51be29729b':0, 'd36d84f51a':0,
                                                  'eb35a5eb6b':1, '289777e76d':1, 'b026b8ee68':0,
                                                  '915be3a7a4':2, '991c4998fb':2,
                                                  'a996720382':3, 'f1fcd26d00':3,
                                                  '242927d0f5':4, 'ba7b390fc4':4})

In [33]:
# one hot encode
dummied_is_durumu = pd.get_dummies(model_df['is_durumu'], prefix='is_durumu')

model_df = pd.concat([model_df, dummied_is_durumu], axis=1)
model_df.drop('is_durumu', axis=1, inplace=True)

In [34]:
# label encode
for column in ['yas', 'kidem_suresi']:
    le = LabelEncoder()
    model_df[column] = le.fit_transform(model_df[column])

In [35]:
# one hot encode
dummied_egitim = pd.get_dummies(full_df['egitim'], prefix='egitim')
model_df = pd.concat([model_df, dummied_egitim], axis=1)

In [36]:
# le3 = LabelEncoder()
# le4 = LabelEncoder()
# le5 = LabelEncoder()

# model_df['islem_adedi_type'] = le3.fit_transform(full_df['islem_adedi_type'])
# model_df['harcama_type'] = le4.fit_transform(full_df['harcama_type'])
# model_df['per_type'] = le5.fit_transform(full_df['per_type'])

In [37]:
# model_df['yas'] = full_df['yas']
# model_df['kidem_suresi'] = full_df['kidem_suresi']

In [38]:
# dummied_meslek_grubu = pd.get_dummies(full_df['meslek_grubu'], prefix='meslek_grubu')
# model_df = pd.concat([model_df, dummied_meslek_grubu], axis=1)

In [39]:
model_df = pd.concat([model_df, full_df.iloc[:,5:]], axis=1)

del full_df
del monthly_exp

In [40]:
for column in ['harcama_type', 'islem_type']:
    le = LabelEncoder()
    model_df[column] = le.fit_transform(model_df[column])

In [41]:
# for column in list(model_df.loc[:,'ay_1_RESTORAN_CATER':'ay_6_KLP_DERNK_SOSY'].columns):
#     model_df[column] = pd.cut(model_df[column], 6)
    
# for column in list(model_df.loc[:,'ay_1':'ay_6'].columns):
#     model_df[column] = pd.cut(model_df[column], 6)

In [42]:
# for column in list(model_df.loc[:,'ay_1_RESTORAN_CATER':'ay_6_KLP_DERNK_SOSY'].columns):
#     le = LabelEncoder()
#     model_df[column] = le.fit_transform(model_df[column])
    
# for column in list(model_df.loc[:,'ay_1':'ay_6'].columns):
#     le = LabelEncoder()
#     model_df[column] = le.fit_transform(model_df[column])

In [43]:
# modele sokulcak verinin son hali
model_df.head()

,yas,kidem_suresi,is_durumu_0,is_durumu_1,is_durumu_2,is_durumu_3,is_durumu_4,egitim_01a4f2c96c,egitim_5f8ca0f83b,egitim_7bb291e291,...,ay_6_KUYUMCU,ay_6_KLP_DERNK_SOSY,harcama_type,ay_1,ay_2,ay_3,ay_4,ay_5,ay_6,islem_type
0,5,2,0,0,1,0,0,0,1,0,...,0.0,0.0,1,-1.593089,0.876684,0.068691,-0.276160,0.508936,0.000000,3
1,5,5,0,0,1,0,0,1,0,0,...,0.0,0.0,3,-1.615986,-0.807993,-0.807993,-1.615986,-1.593089,0.853787,0
2,5,5,1,0,0,0,0,0,0,0,...,0.0,0.0,2,0.000000,-0.807993,0.000000,0.000000,0.000000,0.000000,2
3,4,4,0,0,1,0,0,0,1,0,...,0.0,0.0,2,0.000000,0.000000,0.000000,0.000000,-0.807993,0.000000,2
4,5,4,0,0,0,0,1,0,0,1,...,0.0,0.0,4,-0.762199,0.000000,0.000000,0.000000,0.000000,0.000000,3


In [44]:
# split train ve test
X_train = model_df[:60000]
X_test = model_df[60000:]

In [45]:
# X_test = X_test.reset_index(drop=True)
# X_train = X_train.reset_index(drop=True)

# X_train = X_train.sample(frac=1, random_state=1461)
# y = y.sample(frac=1, random_state=1461)

In [46]:
# scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [48]:
del model_df
# del X_train
# del X_test

# Sampler

In [49]:
# imbalance problem oldugu için oversampling & undersampling & kombinasyonlarını denemek istedim
# problemler şöyle oldu
# - undersampling yapınca çok az veri ile eğitim gerçekleşiyor yetersiz kalıyor
# - oversampling overfitting olma olasılıgı yüksek. Verinin dikkatli kullanılması lazım
# - kombinasyonlarında da overfitting sorunu var, cross validation aldanmamak gerekiyor

# XGBoost, RandomForest, Keras da weight vererek, normal veri üzerinde de denerim. Weight ile daha stabil ve biraz daha iyi sonuçlar elde edilebiliyor

In [50]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import NearMiss, EditedNearestNeighbours
from sklearn.utils import shuffle
from imblearn.under_sampling import TomekLinks

In [51]:
# kendimce undersampling yaptım
deneme_x_1 = X_train.iloc[y[y==1].index]
deneme_y_1 = y[y==1]

sample = y[y==0].sample(2440)
deneme_x_0 = X_train.iloc[sample.index]
deneme_y_0 = sample

deneme_X = pd.concat([deneme_x_1, deneme_x_0], axis=0) 
deneme_y = pd.concat([deneme_y_1, deneme_y_0], axis=0).astype('int32')
#np.concatenate

deneme_y = deneme_y.sample(frac=1, random_state=1461)
deneme_X = deneme_X.sample(frac=1, random_state=1461)

In [ ]:
# imblearn kütüphanesi kullanarak undersampling, oversampling ve kombinasyonlarını yaptım
x_train, x_val, y_train, y_val = train_test_split(X_train, y,
                                                  test_size = .33,
                                                  random_state=14,
                                                 stratify=y)

# oversamplers
ovs = RandomOverSampler(random_state=14)
x_res, y_res = ovs.fit_sample(x_train, y_train)

smk = SMOTETomek(random_state=14)
x_res2, y_res2 = smk.fit_sample(x_train, y_train)

# undersamplers
ous = RandomUnderSampler(random_state=14)
x_res3,y_res3 = ous.fit_sample(X_train, y)

nm = NearMiss()
x_res4,y_res4 = nm.fit_sample(X_train, y)

In [ ]:
# kombinasyon
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.8)

x_overed, y_overed = over.fit_sample(x_train, y_train)
x_res5, y_res5 = under.fit_sample(x_overed, y_overed)

In [ ]:
# kombinasyon
resample = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))

x_res6, y_res6 = resample.fit_sample(x_train, y_train)
y_res6 = y_res6.astype('int')

In [ ]:
# kombinasyon
resample = SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='majority'))

x_res7, y_res7 = resample.fit_sample(x_train, y_train)
y_res7 = y_res7.astype('int')

# ML Approach

In [47]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report, accuracy_score, f1_score, recall_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, StratifiedKFold, learning_curve, cross_validate, RepeatedStratifiedKFold
from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier
import xgboost as xgb
import lightgbm as lgb

In [48]:
def plot_roc_curve(false_positive_rate, true_positive_rate, label=None):
    plt.plot(false_positive_rate, true_positive_rate, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'r', linewidth=4)
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (FPR)', fontsize=16)
    plt.ylabel('True Positive Rate (TPR)', fontsize=16)

In [49]:
def get_pred(model, X, y, X_test):
    model.fit(X, y)
    preds = model.predict(X_test)
    return preds

In [50]:
def submit_csv(model, X, y, X_test, name='submit.csv'):
    
    model.fit(X, y)
    
    preds = model.predict(X_test)
    sub['target'] = preds
    print(Counter(preds))
    sub.to_csv(name, index=False)

In [51]:
def estimate(models, X, y):
    
    X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X, y, test_size=.25, random_state=14)
    
    for model in models:

        model.fit(X_train_s, y_train_s)

        y_scores = model.predict_proba(X_test_s)
        y_scores = y_scores[:,1]

        false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_s, y_scores)

        
        print('-----',model.__class__.__name__,'-----')
        auroc = roc_auc_score(y_test_s, y_scores)
        print(classification_report(y_test_s, model.predict(X_test_s)))
        print("ROC-AUC Score:", auroc)
        print("Confusion Matrix:\n",confusion_matrix(y_test_s,model.predict(X_test_s)),'\n')

        if len(models) == 1: 
            plt.figure(figsize=(14, 7))
            plot_roc_curve(false_positive_rate, true_positive_rate)

In [52]:
# daha karmaşık modeller denemeden önce basit ve farklı modeller üzerinde hızlı bir şekilde iyi skor alablir miyim onu denedim

kf = StratifiedKFold(10, shuffle=True, random_state=14)

m_logreg = LogisticRegression(random_state=14)

m_dtc = DecisionTreeClassifier(random_state=14)

m_ada = AdaBoostClassifier(DecisionTreeClassifier(random_state=14),random_state=14,learning_rate=0.1)

m_rf = RandomForestClassifier(random_state=14)

m_ext = ExtraTreesClassifier(random_state=14)

m_gdb = GradientBoostingClassifier(random_state=14)

m_mlp = MLPClassifier(random_state=14)

m_knn = KNeighborsClassifier()

m_lda = LinearDiscriminantAnalysis()

# models = [m_dtc, m_ada, m_dtc, m_rf, m_ext, m_gdb, m_knn, m_lda]
models = [m_logreg, m_dtc, m_ada, m_rf, m_ext, m_gdb, m_mlp, m_knn, m_lda]

In [53]:
# Models

cv = StratifiedKFold(10, shuffle=True, random_state=42)

rf = RandomForestClassifier(criterion='gini',
                            n_estimators=1750,
                            max_depth=7,
                            min_samples_split=6,
                            min_samples_leaf=6,
                            max_features='auto',
                            oob_score=True,
                            random_state=42,
                            n_jobs=-1,
                            verbose=0)

lg = lgb.LGBMClassifier(max_bin=4,
                        num_iterations=550,
                        learning_rate=0.0114,
                        max_depth=3,
                        num_leaves=7,
                        colsample_bytree=0.35,
                        random_state=42,
                        n_jobs=-1)

xg = xgb.XGBClassifier(
    n_estimators=2800,
    min_child_weight=0.1,
    learning_rate=0.002,
    max_depth=2,
    subsample=0.47,
    colsample_bytree=0.35,
    gamma=0.4,
    reg_lambda=0.4,
    random_state=42,
    n_jobs=-1,
)

logreg = LogisticRegression(n_jobs=-1, solver='newton-cg')

gb = GradientBoostingClassifier(random_state=42)

gnb = GaussianNB()

mlp = MLPClassifier(random_state=42)

estimators = [rf, lg, xg, gb, logreg, gnb, mlp]

In [54]:
def model_check(X, y, estimators, cv):
    model_table = pd.DataFrame()

    row_index = 0
    for est in estimators:

        MLA_name = est.__class__.__name__
        model_table.loc[row_index, 'Model Name'] = MLA_name
        #    model_table.loc[row_index, 'MLA Parameters'] = str(est.get_params())

        cv_results = cross_validate(
            est,
            X,
            y,
            cv=cv,
            scoring='accuracy',
            return_train_score=True,
            n_jobs=-1
        )

        model_table.loc[row_index, 'Train Accuracy Mean'] = cv_results[
            'train_score'].mean()
        model_table.loc[row_index, 'Test Accuracy Mean'] = cv_results[
            'test_score'].mean()
        model_table.loc[row_index, 'Test Std'] = cv_results['test_score'].std()
        model_table.loc[row_index, 'Time'] = cv_results['fit_time'].mean()

        row_index += 1

    model_table.sort_values(by=['Test Accuracy Mean'],
                            ascending=False,
                            inplace=True)

    return model_table

In [ ]:
raw_models = model_check(deneme_X, deneme_y, models, kf)
display(raw_models.style.background_gradient(cmap='summer_r'))

In [ ]:
raw_models = model_check(deneme_X, deneme_y, estimators, cv)
display(raw_models.style.background_gradient(cmap='summer_r'))

In [ ]:
raw_models = model_check(deneme_X, deneme_y, [RFC_best], cv)
display(raw_models.style.background_gradient(cmap='summer_r'))

In [ ]:
# GradientBoosting iyi bir sonuç vermişti bu yüzden parametre tunning yaparak sonuçlarını kontrol ettim
par = {'n_estimators': 100,
   'min_samples_leaf': 125,
   'max_features': 0.5,
   'max_depth': 4,
   'loss': 'exponential',
   'learning_rate': 0.1}
gb = GradientBoostingClassifier(random_state=42)
gb.set_params(**par)
raw_models = model_check(deneme_X, deneme_y, [gb], cv)
display(raw_models.style.background_gradient(cmap='summer_r'))

In [62]:
# RFC Parameters tunning 
RFC = GradientBoostingClassifier(random_state=14)
kf2 = StratifiedKFold(5, shuffle=True, random_state=14)

params = {'loss' : ["deviance",'exponential'],
              'n_estimators' : [100,200,400,600],
              'learning_rate': [0.1,0.3, 0.05, 0.01,1],
              'max_depth': [None,2,3,4],
              'min_samples_leaf': [15,25,50,100,150],
              'max_features': [0.7,0.5,0.2,0.3] 
              }

gsRFC = RandomizedSearchCV(RFC, param_distributions = params,n_iter=5,  cv=kf2, scoring="roc_auc", n_jobs= -1, verbose = 1)

gsRFC.fit(deneme_X, deneme_y)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

NameError: name 'deneme_X' is not defined

In [ ]:
RFC = RandomForestClassifier(random_state=14)

RFC.set_params(**{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': None,
 'max_features': 2,
 'min_samples_leaf': 3,
 'min_samples_split': 2,
 'n_estimators': 300})

ETC = ExtraTreesClassifier(random_state=14)

ETC.set_params(**{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': 1,
 'min_samples_leaf': 3,
 'min_samples_split': 2,
 'n_estimators': 500})

LDA = LinearDiscriminantAnalysis()

KNN = KNeighborsClassifier()

KNN.set_params(**{'algorithm': 'auto',
 'leaf_size': 1,
 'n_neighbors': 10,
 'p': 1,
 'weights': 'uniform'})

GBC = GradientBoostingClassifier(random_state=14)
GBC.set_params(**{'learning_rate': 0.05,
 'loss': 'deviance',
 'max_depth': 4,
 'max_features': 0.1,
 'min_samples_leaf': 50,
 'n_estimators': 200})

print('OK')

In [ ]:
x_t, x_v, y_t, y_v = train_test_split(deneme_X, deneme_y,
                                                  test_size = .33,
                                                  random_state=14,
                                                 stratify=deneme_y)

In [63]:
best_params = {'updater': ['grow_gpu_hist'],
 'tree_method': ['gpu_hist'],
 'subsample': [1.0],
 'predictor': ['gpu_predictor'],
 'n_estimators': [600],
 'min_child_weight': [10],
 'max_depth': [8],
 'learning_rate': [0.05],
 'gamma': [0.5],
 'colsample_bytree': [0.6]}


params = {
        'min_child_weight': [2,3,4,5, 10, 15],
        'gamma': [0.5, 1, 1.5, 2, 3.5, 5],
        'subsample': [0.4, 0.6, 0.8, 1.0],
        'colsample_bytree': [0.4, 0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 6, 8],
        'n_estimators': [400,600,800,1000,1200,1600,2000,2500],
        'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.5, 1, 1.5]
        }

params['tree_method'] = ['gpu_hist']
params['updater'] = ['grow_gpu_hist']
params['predictor'] = ['gpu_predictor']

class_weight = int(y.value_counts()[0] / y.value_counts()[1])
# class_weight = 1
xgb = XGBClassifier(objective='binary:logistic',
                    silent=True, nthread=1,eval_metric='auc',verbosity=0,scale_post_weight=class_weight)

folds = 3
param_comb = 15
SEED = 1461

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = SEED)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=-1, cv=skf.split(X_train_scaled, y), verbose=3, random_state=SEED)

random_search.fit(X_train_scaled, y)
print('OK')

Fitting 3 folds for each of 15 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  2.9min finished


OK


In [64]:
print(random_search.cv_results_['split0_test_score'])
print(random_search.cv_results_['split1_test_score'])
print(random_search.cv_results_['split2_test_score'])
print(random_search.cv_results_['mean_test_score'])
print(random_search.cv_results_['std_test_score'])
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)

[0.76072119 0.73347694 0.76660531 0.76573295 0.64852233 0.75100277
 0.62057194 0.64723043 0.76954158 0.76938154 0.64703952 0.76684228
 0.73768403 0.66562272 0.7605624 ]
[0.75762712 0.72641666 0.76323709 0.76327151 0.63424853 0.74957182
 0.6239143  0.62852789 0.76568006 0.76451992 0.62888833 0.76353986
 0.74421027 0.65000961 0.75901038]
[0.7601529  0.7358902  0.75895046 0.75566941 0.65431348 0.75478082
 0.6183301  0.65545653 0.76112755 0.7617596  0.6595446  0.76184128
 0.74358722 0.65015469 0.75416045]
[0.75950041 0.73192793 0.76293095 0.76155796 0.64569478 0.75178514
 0.62093878 0.64373828 0.76544973 0.76522036 0.64515748 0.76407447
 0.74182717 0.65526234 0.75791108]
[0.00134478 0.00401966 0.00313257 0.00428337 0.00843196 0.00219735
 0.00229445 0.01126748 0.00343887 0.00315081 0.01258593 0.00207635
 0.00294067 0.00732614 0.00272673]

 Best normalized gini score for 3-fold search with 15 parameter combinations:
0.530899464351678


In [66]:
sub['target'] = random_search.predict_proba(X_test_scaled)[:,1]
sub.to_csv('deneme2.csv', index=False)

In [ ]:
# print(confusion_matrix(y_v, random_search.predict(x_v)))
# print(classification_report(y_v, random_search.predict(x_v)))

model = random_search
final_preds = model.predict_proba(X_train)[:,1]
fpr, tpr, thresholds = roc_curve(y, final_preds)
# thresholds
accuracy_ls = []

for thres in thresholds:
    y_pred = np.where(final_preds>thres,1,0)
    accuracy_ls.append(roc_auc_score(y, y_pred))
    
accuracy_ls = pd.concat([pd.Series(thresholds), pd.Series(accuracy_ls)],
                        axis=1)
accuracy_ls.columns = ['thresholds', 'roc']
accuracy_ls.sort_values(by='roc', ascending=False, inplace=True)
accuracy_ls.head()

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train_scaled, label=y)
dtest = xgb.DMatrix(X_test_scaled)

paramlar = {'updater': 'grow_gpu_hist',
 'tree_method': 'gpu_hist',
 'subsample': 1.0,
 'predictor': 'gpu_predictor',
 'n_estimators': 800,
 'min_child_weight': 3,
 'max_depth': 4,
 'learning_rate': 0.1,
 'gamma': 5,
 'colsample_bytree': 0.8}

params['eval_metric'] = 'auc'
params['objective'] = 'binary:logistic'
params['scale_pos_weight'] = np.sqrt(int(y.value_counts()[0] / y.value_counts()[1]))

bst = xgb.train(paramlar, dtrain, 35)

In [ ]:
final_preds = bst.predict(dtrain)
fpr, tpr, thresholds = roc_curve(y, final_preds)
# thresholds
accuracy_ls = []
for thres in thresholds:
    y_pred = np.where(final_preds>thres,1,0)
    accuracy_ls.append(roc_auc_score(y, y_pred))
    
accuracy_ls = pd.concat([pd.Series(thresholds), pd.Series(accuracy_ls)],
                        axis=1)
accuracy_ls.columns = ['thresholds', 'roc']
accuracy_ls.sort_values(by='roc', ascending=False, inplace=True)
accuracy_ls.head()

In [ ]:
# submision yaptıgım notebook şansız olaydan dolayı kaybettiğim için burada submit dosyam bulunmuyor
# XGBoost'u kendi yaptıgım undersampling verisi ile eğittim ve hyperparameter tunning ile kendi skorumu elde ettim

# proba yerine 0 ve 1 ler ile submit ettiğim için daha düşük aldıgımı düşünüyorum cv de 77-78 gibi auc-roc score elde ediliyor